## Brightness function

In [1]:
def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

In [39]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
profileface_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_profileface.xml')
eye_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_eye_tree_eyeglasses.xml')
###########
classNames =[]
classFile = 'coco.names'
with open(classFile,'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')

configPath = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
weightsPath = 'frozen_inference_graph.pb'
classes=list(range(41,62))
net = cv2.dnn_DetectionModel(weightsPath,configPath)
net.setInputSize(320,320)
net.setInputScale(1.0/127.5)
net.setInputMean((127.5,127.5,127.5))
net.setInputSwapRB(True)
##########
while True:
    ret, frame = cap.read()
    
    classIds, confs, bbox = net.detect(frame,confThreshold=0.5)
    #print(classIds,bbox)
    if len(classIds) != 0 :
        for classId,confidence,box in zip(classIds.flatten(),confs.flatten(),bbox):
            if (classId in classes)or(classId==77) : # detect cell phone and food ONLY
                cv2.rectangle(frame,box,color=(0,255,0),thickness=2)
                cv2.putText(frame,classNames[classId-1].upper()+': '+str(round(confidence*100))+'%',(box[0]+10,box[1]+30),cv2.FONT_HERSHEY_COMPLEX,.8,(0,255,0),2)

    img = increase_brightness(frame, 50)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    face2 = profileface_cascade.detectMultiScale(gray, 1.3, 5)
    if len(faces)==0 and len(face2)==0 :
        cv2.putText(frame,'NO FACE',(50,50),cv2.FONT_HERSHEY_COMPLEX,.8,(0,0,255),2)
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 5)
        roi_gray = gray[y:y+w, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray, 1.3, 5)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 5)
        if len(eyes) == 0:
            cv2.putText(frame,'No eyes!!!',(50,50),cv2.FONT_HERSHEY_COMPLEX,.8,(0,255,0),2)
        else:
            cv2.putText(frame,'eyes!!!',(50,50),cv2.FONT_HERSHEY_COMPLEX,.8,(0,255,0),2)

    if len(faces)==0:
        for (x, y, w, h) in face2:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 5)
            roi_gray = gray[y:y+w, x:x+w]
            roi_color = frame[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_gray, 1.3, 5)
            for (ex, ey, ew, eh) in eyes:
                cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 5)
            if len(eyes) == 0:
                cv2.putText(frame,'No eyes!!!',(50,50),cv2.FONT_HERSHEY_COMPLEX,.8,(0,255,0),2)
            else:
                cv2.putText(frame,'eyes!!!',(50,50),cv2.FONT_HERSHEY_COMPLEX,.8,(0,255,0),2)
    
    cv2.imshow('frame', frame)
    if cv2.waitKey(100) == 27:
        break

cap.release()
cv2.destroyAllWindows()